### change path below to your gdrive path where folder is 

In [ ]:
%cd '/content/drive/MyDrive/Pavement_Data Science/Model_1/DSPS'

/content/drive/MyDrive/Pavement_Data Science/Model_1/DSPS


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Uncomment the lines below to download training data
!gdown 'https://drive.google.com/uc?id=1I1onAA61iui4LjOxejkxioaBjU1Ic_6X'
!unzip 'training_data.zip' -d './cvat'

Downloading...
From: https://drive.google.com/uc?id=1I1onAA61iui4LjOxejkxioaBjU1Ic_6X
To: /content/drive/My Drive/Pavement_Data Science/Model_1/DSPS/training_data.zip
100% 259M/259M [00:01<00:00, 131MB/s]
Archive:  training_data.zip
replace ./cvat/td3/annotations/instances_default.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./cvat/td3/annotations/instances_default.json  
replace ./cvat/td3/images/0 (29).JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./cvat/td3/images/0 (29).JPG  
  inflating: ./cvat/td3/images/0 (128).JPG  
  inflating: ./cvat/td3/images/0 (65).JPG  
  inflating: ./cvat/td3/images/0 (107).JPG  
  inflating: ./cvat/td3/images/0 (5).JPG  
  inflating: ./cvat/td3/images/0 (14).JPG  
  inflating: ./cvat/td3/images/0 (45).JPG  
  inflating: ./cvat/td3/images/0 (83).JPG  
  inflating: ./cvat/td3/images/0 (31).JPG  
  inflating: ./cvat/td3/images/0 (8).JPG  
  inflating: ./cvat/td3/images/0 (119).JPG  
  inflating: ./cvat/td3/images/0 (115).JPG  
  i

In [ ]:
# cvat folder should contain 3 folders: td1, td2, td3
!ls cvat 

td1  td2  td3


## Uncomment and run codes below if this is the first time running this notebook

In [ ]:
!pip install -U PyYAML

In [ ]:
!pip install -qr dspYolov5/requirements.txt  # install dependencies for yolov5

## Uncomment block of codes in cell below to convert cvat annotations to yolo annotations

In [ ]:
import os
from dsps.simple_fxn import cvat2yolo
# location of cvat annotations
annotation_folder = 'cvat/'
output_path = 'data/training'
val_path = 'data/validation'
for annts in os.listdir(annotation_folder):
  json_file = os.path.join(annotation_folder,annts,'annotations','instances_default.json')
  img_path = os.path.join(annotation_folder,annts,'images')
  if(os.path.isfile(json_file)):
    cvat2yolo(annts,json_file,img_path, output_path, val_path)

{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['LONGITUDINAL', 'TRANSVERSE', 'BLOCK', 'ALLIGATOR', 'PATCHING', 'SEALING', 'MANHOLE']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['TRANSVERSE', 'PATCHING', 'LONGITUDINAL', 'BLOCK', 'ALLIGATOR', 'SEALING']
{1: 'ALLIGATOR', 2: 'BLOCK', 3: 'TRANSVERSE', 4: 'PATCHING', 5: 'SEALING', 6: 'LONGITUDINAL', 7: 'MANHOLE'}
total number of classes: 7
['PATCHING', 'BLOCK', 'ALLIGATOR', 'SEALING', 'LONGITUDINAL', 'TRANSVERSE', 'MANHOLE']


## Train a new model

### If not running on GPU, change runtime to GPU
________

In [ ]:
import json
import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla K80)


### **Change class labels and number of classes**: 
* Edit dspYolov5/data/dsps.yaml file 
### **Change model configuration**: 
* Edit dspYolov5/models/dsps_model.yaml

### run the code below. You may get an error the first time. 

---

restart the runtime and run again. 

stop cell after total epoch have elapsed. 

In [ ]:
!python -i dspYolov5/train_dsps.py --data dspYolov5/data/dsps.yaml --cfg dspYolov5/models/dsps_model.yaml --batch-size 16 --device 0

100% 755k/755k [00:00<00:00, 18.8MB/s]
train: weights=yolov5s.pt, cfg=dspYolov5/models/dsps_model.yaml, data=dspYolov5/data/dsps.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=150, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=weights/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=150
github: Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
YOLOv5 🚀 2022-1-10 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_

### Evaluate and Visualize Model Performance

#### change weights path in the following cell: replace exp12 in the weights path with your corresponding folder-- 'weights/train/exp12/weights/best.pt'

In [ ]:
## Uncomment the lines below to download training data
!gdown 'https://drive.google.com/uc?id=1TtDGMgTyrH6bclBf76ggZk28N9aalOzc'
!unzip 'dsps_test.zip' -d './data/test_images'

Downloading...
From: https://drive.google.com/uc?id=1TtDGMgTyrH6bclBf76ggZk28N9aalOzc
To: /content/drive/MyDrive/Pavement_Data Science/Model_1/DSPS/dsps_test.zip
100% 330M/330M [00:02<00:00, 128MB/s]
Archive:  dsps_test.zip
replace ./data/test_images/images/5 (7).JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ./data/test_images/images/5 (7).JPG  
replace ./data/test_images/images/8 (1).JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace ./data/test_images/images/8 (1).JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./data/test_images/images/8 (1).JPG  
  inflating: ./data/test_images/images/6 (63).JPG  
  inflating: ./data/test_images/images/8 (44).JPG  
  inflating: ./data/test_images/images/4 (66).JPG  
  inflating: ./data/test_images/images/1 (33).JPG  
  inflating: ./data/test_images/images/8 (2).JPG  
  inflating: ./data/test_images/images/5 (53).JPG  
  inflating: ./data/test_images/images/1 (35).JPG  
  inflating: ./data/test_im

In [ ]:
## create results folder if it doen't exist

In [ ]:
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp4/weights/last.pt' --img 640 --conf 0.25 --source data/test_images/images

detect: weights=['weights/train/exp11/weights/last.pt'], source=data/test_images/images, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /content/drive/MyDrive/Pavement_Data Science/Model_1/DSPS/requirements.txt not found, check failed.
YOLOv5 🚀 2022-1-10 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 224 layers, 7070092 parameters, 0 

### Upload submission.json file in the "results" folder to leaderboard

In [ ]:
## sample submission file content
# [{"1 (134).JPG": [[3, 365, 585, 895, 697], [2, 41, 588, 917, 2000]]}, {"1 (154).JPG": [[1, 8, 1622, 429, 2007], [6, 24, 0, 157, 867]]}]
# [3, 365, 585, 895, 697] --> ['class','x1','y1','x2','y2']
# x1,y1 - top left corner of bounding box
# x2,y2 _ bottom right corner of bounding box

### Auto Annotate and re-upload  to cvat 

In [ ]:
!python 'dspYolov5/detect_dsps.py' --weights 'weights/train/exp/weights/best.pt' --img 640 --conf 0.25 --source data/auto_annotate/ --nosave

In [ ]:
## this will create a cvat compatible annotation file 
img_path = 'data/test_images'
csv_path = 'results/submission.csv'
output_path = 'results/instances_default.json'
csv2cvat(csv_path, img_path,output_path)